## Neural Networks를 이용한 당뇨병 예측

## 0. 라이브러리

In [ ]:
import sys, os

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns




## 1. 데이터

* https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database
* 이 데이터셋은 미국 국립당뇨병 및 소화기 및 신장 질환 연구소(National Institute of Diabetes and Digestive and Kidney Diseases)에서 제공한 데이터입니다.
* 이 데이터셋의 목적은 데이터셋에 포함된 특정 진단 측정값을 기반으로 환자가 당뇨병을 가지고 있는지 여부를 진단적으로 예측하는 것이며, 데이터셋의 모든 환자는 최소나이가 21세인 피마 인디언 유산의 여성들입니다.

### 1) 다운로드/로드

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("uciml/pima-indians-diabetes-database")

# print("Path to dataset files:", path)

In [7]:
DATASET_PATH = '../../dataset/diabetes.csv'
df = pd.read_csv(DATASET_PATH)
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


### 2) 탐색

#### 컬럼 정보

* Pregnancies: 임신 횟수
* Glucose: 혈장 내 포도당 농도
* BloodPressure: 이완기 혈압 (mm Hg)
* SkinThickness: 삼두근 피부접힘 두께 (mm)
* Insulin: 혈청 인슐린 농도 (mu U/ml)
* BMI: 체질량 지수
* DiabetesPedigreeFunction: 유전적 관계 정보
* Age: 나이
* Outcome: 클래스 (0은 정상, 1은 당뇨병)

In [9]:
# 기초 통계 확인
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Pregnancies,768.0,3.845052,3.369578,0.000,1.00000,3.0000,6.00000,17.00
Glucose,768.0,120.894531,31.972618,0.000,99.00000,117.0000,140.25000,199.00
BloodPressure,768.0,69.105469,19.355807,0.000,62.00000,72.0000,80.00000,122.00
SkinThickness,768.0,20.536458,15.952218,0.000,0.00000,23.0000,32.00000,99.00
Insulin,768.0,79.799479,115.244002,0.000,0.00000,30.5000,127.25000,846.00
BMI,768.0,31.992578,7.884160,0.000,27.30000,32.0000,36.60000,67.10
DiabetesPedigreeFunction,768.0,0.471876,0.331329,0.078,0.24375,0.3725,0.62625,2.42
Age,768.0,33.240885,11.760232,21.000,24.00000,29.0000,41.00000,81.00
Outcome,768.0,0.348958,0.476951,0.000,0.00000,0.0000,1.00000,1.00


In [10]:
# 결측치 확인
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [ ]:
# 최솟값 확인 -> 이상 데이터 (Pregnancies, Outcome을 제외하고는 결측값이 있을 수 있음)
def highlight_min(s):
    is_max = s==s.min()
    return ['background-color: limegreen' if v else '' for v in is_max]

df.describe().T.style.apply(highlight_min, subset=['min'])

,count,mean,std,min,25%,50%,75%,max
Pregnancies,768.000000,3.845052,3.369578,0.000000,1.000000,3.000000,6.000000,17.000000
Glucose,768.000000,120.894531,31.972618,0.000000,99.000000,117.000000,140.250000,199.000000
BloodPressure,768.000000,69.105469,19.355807,0.000000,62.000000,72.000000,80.000000,122.000000
SkinThickness,768.000000,20.536458,15.952218,0.000000,0.000000,23.000000,32.000000,99.000000
Insulin,768.000000,79.799479,115.244002,0.000000,0.000000,30.500000,127.250000,846.000000
BMI,768.000000,31.992578,7.884160,0.000000,27.300000,32.000000,36.600000,67.100000
DiabetesPedigreeFunction,768.000000,0.471876,0.331329,0.078000,0.243750,0.372500,0.626250,2.420000
Age,768.000000,33.240885,11.760232,21.000000,24.000000,29.000000,41.000000,81.000000
Outcome,768.000000,0.348958,0.476951,0.000000,0.000000,0.000000,1.000000,1.000000


In [19]:
# 데이터 정보 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


### 3) 전처리

In [21]:
# 결측치 처리 (값이 0일 수 없는 컬럼)
df[['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']] = df[['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']].replace(0, np.nan)

In [ ]:
# 결측값 비율 확인
percent_missing = df.isnull().mean().round(4)*100
percent_missing

Pregnancies                  0.00
Glucose                      0.65
BloodPressure                4.56
SkinThickness               29.56
Insulin                     48.70
BMI                          1.43
DiabetesPedigreeFunction     0.00
Age                          0.00
Outcome                      0.00
dtype: float64